In [2]:
!pip uninstall openai


In [2]:
pip install python-dotenv


  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)
Note: you may need to restart the kernel to use updated packages.


In [6]:
import openai
from openai import OpenAI

In [5]:
# import os
# from openai import OpenAI

# client = OpenAI(
#     api_key=os.getenv("OPENAI_API_KEY"),  # This is the default and can be omitted
# )

# prompt = "make simple brief description of the given 3d model, the about the object features with out describing the colors or textures, just the appearance of the mesh"

# with open("1a04e3eab45ca15dd86060f189eb133\\1a04e3eab45ca15dd86060f189eb133_45.png", "rb") as image_file:
#     img_b64_str = base64.b64encode(image_file.read()).decode("utf-8")

# response = client.chat.completions.create(
#     model="gpt-4o-mini",
#     messages=[
#         {
#             "role": "user",
#             "content": [
#                 {"type": "text", "text": prompt},
#                 {
#                     "type": "image_url",
#                     "image_url": {"url": f"data:{"image/png"};base64,{img_b64_str}"},
#                 },
#             ],
#         }
#     ],
# )



In [9]:
# response.choices[0].message.content

In [ ]:
import os
import base64
from dotenv import load_dotenv

# Load variables from .env file
load_dotenv()

# Set your API key
client = OpenAI(
    api_key="",  # This is the default and can be omitted
)

def get_caption_for_image(image_path):
    """
    Reads the image file, encodes it to base64, sends an API request to generate a caption,
    and returns the caption as text.
    """
    with open(image_path, "rb") as f:
        img_b64_str = base64.b64encode(f.read()).decode("utf-8")
    
    # prompt = "Describe the 3D model focusing only on its geometric shape and structure. Do not mention colors, textures, or any material properties—only the form and features of the mesh."

    prompt = "Provide a single concise, technical description less than 50 words of the 3D model's structure. Describe its geometric attributes—such as the number and arrangement of wings, the overall body shape, and other key structural details—without mentioning any colors, textures, or decorative elements."

    response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": prompt},
                {
                    "type": "image_url",
                    "image_url": {"url": f"data:{"image/png"};base64,{img_b64_str}"},
                },
            ],
        }
    ],
)
    
    # Extract and return just the text content of the response message
    caption = response.choices[0].message.content
    return caption

def process_folder_structure(root_dir):
    """
    Iterates through the folder structure under root_dir, finds every image file,
    generates a caption using the OpenAI API, and saves the caption to a text file
    in the same folder.
    """
    for dirpath, dirnames, filenames in os.walk(root_dir):
        for file in filenames:
            # Check if file is an image (you can add more extensions as needed)
            if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                image_path = os.path.join(dirpath, file)
                print(f"Processing {image_path}...")
                
                try:
                    caption = get_caption_for_image(image_path)
                    # Create a text file with the same base name and a _caption.txt suffix
                    caption_file = os.path.splitext(file)[0] + "_caption.txt"
                    caption_path = os.path.join(dirpath, caption_file)
                    
                    with open(caption_path, "w", encoding="utf-8") as f:
                        f.write(caption)
                    
                    print(f"Caption saved for {file} in {caption_path}")
                except Exception as e:
                    print(f"Error processing {image_path}: {e}")

# Example usage:
root_directory = "E:\\Danilka\\AutoMesh\\preprocessed-airplane-captions-sample\\non_captioned_models"  # Update this to your folder path
process_folder_structure(root_directory)


Processing E:\Danilka\AutoMesh\preprocessed-airplane-captions-sample\non_captioned_models\15442b53631b7e88232438a710acf1da\15442b53631b7e88232438a710acf1da_45.png...
Caption saved for 15442b53631b7e88232438a710acf1da_45.png in E:\Danilka\AutoMesh\preprocessed-airplane-captions-sample\non_captioned_models\15442b53631b7e88232438a710acf1da\15442b53631b7e88232438a710acf1da_45_caption.txt
Processing E:\Danilka\AutoMesh\preprocessed-airplane-captions-sample\non_captioned_models\157a81baeb10914566cf1b4a8fc3914e\157a81baeb10914566cf1b4a8fc3914e_45.png...
Caption saved for 157a81baeb10914566cf1b4a8fc3914e_45.png in E:\Danilka\AutoMesh\preprocessed-airplane-captions-sample\non_captioned_models\157a81baeb10914566cf1b4a8fc3914e\157a81baeb10914566cf1b4a8fc3914e_45_caption.txt
Processing E:\Danilka\AutoMesh\preprocessed-airplane-captions-sample\non_captioned_models\157bb84c08754307dff9b4d1071b12d7\157bb84c08754307dff9b4d1071b12d7_45.png...
Caption saved for 157bb84c08754307dff9b4d1071b12d7_45.png in

KeyboardInterrupt: 

# Seperate captiopned from non captioned

In [1]:
import os
import shutil

def has_caption_file(folder_path, caption_suffix="_caption.txt"):
    """
    Checks if a folder contains any file with the given caption suffix.
    """
    for filename in os.listdir(folder_path):
        if filename.endswith(caption_suffix):
            return True
    return False

def separate_folders(root_folder, captioned_dest, skipped_dest, caption_suffix="_caption.txt"):
    """
    Iterates through each subfolder in root_folder. If the subfolder contains a caption file
    and a folder with the same name already exists in captioned_dest, move it to skipped_dest;
    otherwise, move it to captioned_dest.
    """
    # Ensure destination folders exist
    os.makedirs(captioned_dest, exist_ok=True)
    os.makedirs(skipped_dest, exist_ok=True)
    
    # List all items in the root folder
    for item in os.listdir(root_folder):
        item_path = os.path.join(root_folder, item)
        
        # Only consider subfolders
        if os.path.isdir(item_path):
            if has_caption_file(item_path, caption_suffix):
                dest_path = os.path.join(captioned_dest, item)
                
                if os.path.exists(dest_path):
                    # Move to skipped_dest if folder already exists in captioned_dest
                    skipped_path = os.path.join(skipped_dest, item)
                    print(f"Folder '{item}' already exists in '{captioned_dest}'. Moving to '{skipped_dest}'.")
                    shutil.move(item_path, skipped_path)
                else:
                    print(f"Moving '{item}' to captioned folder.")
                    shutil.move(item_path, dest_path)
            else:
                print(f"'{item}' does not have a caption file. Leaving it in place.")

# Example usage:
root_folder = "E:/Danilka/AutoMesh/preprocessed-airplane-captions-sample/non_captioned_models"
captioned_dest = "E:/Danilka/AutoMesh/preprocessed-airplane-captions-sample/captioned_models"
skipped_dest = "E:/Danilka/AutoMesh/preprocessed-airplane-captions-sample/skipped"

separate_folders(root_folder, captioned_dest, skipped_dest)


Moving '10155655850468db78d106ce0a280f87' to captioned folder.
Moving '117830993cc5887726587cb13c78fb9b' to captioned folder.
Moving '117861b9ebca4f17c69cb28fb4b4d257' to captioned folder.
Moving '1203825bf97bc3524722e1824a086fad' to captioned folder.
Moving '122776d17b6a118086da73d36506db6f' to captioned folder.
Moving '122963149f6a04272620819ddac40644' to captioned folder.
Moving '124062ccaeae95085e9e2656aff7dd5b' to captioned folder.
Moving '125417c3fa9bc1b130f57e42c77d3894' to captioned folder.
Moving '1286826ff37699a1a0d713eb26379316' to captioned folder.
Moving '130422d62194cf026c8067e570d6af71' to captioned folder.
Moving '130934b3dd2fddfaaf4f36f817f09501' to captioned folder.
Moving '132111c575000a3c146e44cc99f07c52' to captioned folder.
Moving '133937bd45f953748be6919d4632fec1' to captioned folder.
Moving '1367266dc0a07c925d4533028830a79b' to captioned folder.
Moving '1385305478499e6c1e06124bec25a766' to captioned folder.
Moving '143326cbe409ca9f62048be44e1fa435' to captioned 

In [ ]:
import os
import base64
from dotenv import load_dotenv
import time
# Load variables from .env file
load_dotenv()

# Set your API key
client = OpenAI(
    # api_key="",  # This is the default and can be omitted
)

def get_caption_for_image(image_path):
    """
    Reads the image file, encodes it to base64, sends an API request to generate a caption,
    and returns the caption as text.
    """
    with open(image_path, "rb") as f:
        img_b64_str = base64.b64encode(f.read()).decode("utf-8")
    
    # prompt = "Describe the 3D model focusing only on its geometric shape and structure. Do not mention colors, textures, or any material properties—only the form and features of the mesh."

    prompt = "Provide a single concise, technical description less than 50 words of the 3D model's structure. Describe its geometric attributes—such as the number and arrangement of wings, the overall body shape, and other key structural details—without mentioning any colors, textures, or decorative elements."

    response = client.chat.completions.create(
    model="gpt-4o-mini",
    # model="gpt-4o",
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": prompt},
                {
                    "type": "image_url",
                    "image_url": {"url": f"data:{"image/png"};base64,{img_b64_str}"},
                },
            ],
        }
    ],
)
    
    # Extract and return just the text content of the response message
    caption = response.choices[0].message.content
    # time.sleep(20)
    return caption

def process_folder_structure(root_dir):
    """
    Iterates through the folder structure under root_dir, finds every image file,
    generates a caption using the OpenAI API, and saves the caption to a text file
    in the same folder.
    """
    for dirpath, dirnames, filenames in os.walk(root_dir):
        for file in filenames:
            # Check if file is an image (you can add more extensions as needed)
            if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                image_path = os.path.join(dirpath, file)
                print(f"Processing {image_path}...")
                
                try:
                    caption = get_caption_for_image(image_path)
                    # Create a text file with the same base name and a _caption.txt suffix
                    caption_file = os.path.splitext(file)[0] + "_caption.txt"
                    caption_path = os.path.join(dirpath, caption_file)
                    
                    with open(caption_path, "w", encoding="utf-8") as f:
                        f.write(caption)
                    
                    print(f"Caption saved for {file} in {caption_path}")
                except Exception as e:
                    print(f"Error processing {image_path}: {e}")

# Example usage:
root_directory = "E:\\Danilka\\AutoMesh\\preprocessed-airplane-captions-sample\\non_captioned_models"  # Update this to your folder path
process_folder_structure(root_directory)


Processing E:\Danilka\AutoMesh\preprocessed-airplane-captions-sample\non_captioned_models\10155655850468db78d106ce0a280f87\10155655850468db78d106ce0a280f87_45.png...


In [36]:
import os
import shutil

def has_caption_file(folder_path, caption_suffix="_caption.txt"):
    """
    Checks if a folder contains any file with the given caption suffix.
    """
    for filename in os.listdir(folder_path):
        if filename.endswith(caption_suffix):
            return True
    return False

def separate_folders(root_folder, captioned_dest, skipped_dest, caption_suffix="_caption.txt"):
    """
    Iterates through each subfolder in root_folder. For each subfolder:
      - If a folder with the same name already exists in captioned_dest:
          - If that folder contains a caption file, skip processing.
          - If it does not contain a caption file, move the subfolder to skipped_dest.
      - If no folder with the same name exists in captioned_dest:
          - If the subfolder contains a caption file, move it to captioned_dest.
          - Otherwise, leave it in place.
    """
    # Ensure destination folders exist
    os.makedirs(captioned_dest, exist_ok=True)
    os.makedirs(skipped_dest, exist_ok=True)
    
    # List all items in the root folder
    for item in os.listdir(root_folder):
        item_path = os.path.join(root_folder, item)
        
        # Only process subfolders
        if os.path.isdir(item_path):
            dest_path = os.path.join(captioned_dest, item)
            if os.path.exists(dest_path):
                # Folder already exists in captioned_dest
                if has_caption_file(dest_path, caption_suffix):
                    print(f"Folder '{item}' already exists in '{captioned_dest}' with caption. Skipping.")
                    skipped_path = os.path.join(skipped_dest, item)
                    shutil.move(item_path, skipped_path)
                else:
                    # Folder exists but without a caption file, so move it to skipped_dest
                    skipped_path = os.path.join(skipped_dest, item)
                    print(f"Folder '{item}' exists in '{captioned_dest}' but lacks caption. Moving to '{skipped_dest}'.")
                    shutil.move(item_path, skipped_path)
            else:
                # Folder does not exist in captioned_dest; check if it contains a caption file.
                if has_caption_file(item_path, caption_suffix):
                    print(f"Moving '{item}' to captioned folder.")
                    shutil.move(item_path, dest_path)
                else:
                    print(f"'{item}' does not have a caption file. Leaving it in place.")

# Example usage:
root_folder = "E:/Danilka/AutoMesh/preprocessed-airplane-captions-sample/non_captioned_models"
captioned_dest = "E:/Danilka/AutoMesh/preprocessed-airplane-captions-sample/captioned_models"
skipped_dest = "E:/Danilka/AutoMesh/preprocessed-airplane-captions-sample/skipped"

separate_folders(root_folder, captioned_dest, skipped_dest)


'10155655850468db78d106ce0a280f87' does not have a caption file. Leaving it in place.
Folder '1021a0914a7207aff927ed529ad90a11' already exists in 'E:/Danilka/AutoMesh/preprocessed-airplane-captions-sample/captioned_models' with caption. Skipping.
Folder '1026dd1b26120799107f68a9cb8e3c' already exists in 'E:/Danilka/AutoMesh/preprocessed-airplane-captions-sample/captioned_models' with caption. Skipping.
Folder '103c9e43cdf6501c62b600da24e0965' already exists in 'E:/Danilka/AutoMesh/preprocessed-airplane-captions-sample/captioned_models' with caption. Skipping.
Folder '105f7f51e4140ee4b6b87e72ead132ed' already exists in 'E:/Danilka/AutoMesh/preprocessed-airplane-captions-sample/captioned_models' with caption. Skipping.
Folder '1066b65c30d153e04c3a35cee92bb95b' already exists in 'E:/Danilka/AutoMesh/preprocessed-airplane-captions-sample/captioned_models' with caption. Skipping.
Folder '106dfe858cb8fbc2afc6b80d80a265ab' already exists in 'E:/Danilka/AutoMesh/preprocessed-airplane-captions-